In [110]:
import json
import inspect
from openai import OpenAI
from typing import get_type_hints

from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from rich import print_json
import rich
import instructor

In [116]:
class FunctionInfo(BaseModel):
    tool_name: str
    tool_inputs: dict

class Functions(BaseModel):
    tool_names: Optional[List[str]] = Field(description="Names of the tools required to execute the user query in the order of execution if user query is a task", default=None)
    parameters: Optional[Dict[str, dict]] = Field(description="input parameters of tools applicable to user query in the format key is tool name value is a dictionary of parameter names and values", default=None)

    


In [117]:
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

def generate_full_completion(model: str, prompt: str, **kwargs) -> dict:
    try:
        # Using patched OpenAI API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            response_model=Functions,
        )
        return response.dict()
    except Exception as err:
        return {"error": f"API call error: {str(err)}"}

def get_type_name(t):
    name = str(t)
    if "list" in name or "dict" in name:
        return name
    else:
        return t.__name__

def function_to_json(func):
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    function_info = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": {"type": "object", "properties": {}},
        "returns": type_hints.get("return", "void").__name__,
    }

    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    return json.dumps(function_info, indent=2)



In [118]:
def file_complaint(data: dict) -> dict:
    """
    Function to file a complaint with the following values inside the data dictionary:
    - name: Name of the person filing the complaint
    - username: Username of the person filing the complaint
    - description: Description of the complaint
    - mobile_number: Mobile number of the person filing the complaint
    """
    # TODO - Implement this function

def search_complaints(data: dict) -> dict:
    """
    Function to search complaints with the following values inside the data dictionary:
    - username: Username of the person searching the complaint
    - mobile_number: Mobile number of the person searching the complaint
    """
    # TODO - Implement this function

def find_city(data: dict) -> dict:
    """
    Function to find city with the following values inside the data dictionary:
    - city_name: Name of the city to search
    """
    # TODO - Implement this function

In [127]:
def main():
    GPT_MODEL = "llama2"

    prompts = [
        "I want to file a complaint in kormangala Bangalore. My name is Gopi and my username is gopi123. The complaint is regarding the garbage collection in my area. The service code is 1234. My mobile number is 1234567890",
        "Check if Kormangala is a city in Bangalore.",
        "Search status of my complaint. My username is gopi123 and my mobile number is 1234567890. My name is Gopi.",
        "Hello how are you ?" 
    ]

    base_prompt = f"""Think step by step. You have the following tools available. 
    Select only tools that are 100% match with the user query.
    Select the parameters for the tool only if it is available in the user query. Do not try to generate parameters.
    Return the selected tool names and input parameters associated with each tool name.
    available tools:\n
    """

    base_prompt = (
        f"{base_prompt}\n"
        f"{function_to_json(file_complaint)}\n"
        f"{function_to_json(search_complaints)}\n"
        f"{function_to_json(find_city)}\n"
        f"User Query:"
    )

    for prompt in prompts:
        # Generate completion
        question = f"{base_prompt} {prompt}"
        print(f"❓User Query: {prompt}")
        response = generate_full_completion(GPT_MODEL, question)
        try:
            tidy_response =response.get("response", response)
            print_json(data=tidy_response)
        except Exception as e:
            print(e)
            print(f"❌ Unable to decode JSON. {response}")

main()

❓User Query: I want to file a complaint in kormangala Bangalore. My name is Gopi and my username is gopi123. The complaint is regarding the garbage collection in my area. The service code is 1234. My mobile number is 1234567890


/var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/ipykernel_8400/832746851.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  return response.dict()


{
  "tool_names": [
    "file_complaint"
  ],
  "parameters": {
    "data": {
      "name": "Gopi",
      "username": "gopi123",
      "description": "Regarding garbage collection in area",
      "mobile_number": "1234567890"
    }
  }
}

❓User Query: Check if Kormangala is a city in Bangalore.


{
  "error": "API call error: 2 validation errors for Functions\nparameters.username\n  Input should be an object [type=dict_type, input_value='userquery', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.4/v/dict_type\nparameters.mobile_number\n  Input should be an object [type=dict_type, input_value='1234567890', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.4/v/dict_type"
}

❓User Query: Search status of my complaint. My username is gopi123 and my mobile number is 1234567890. My name is Gopi.


{
  "tool_names": [
    "search_complaints"
  ],
  "parameters": {
    "data": {
      "username": "gopi123",
      "mobile_number": "1234567890",
      "name": "Gopi"
    }
  }
}

❓User Query: Hello how are you ?


{
  "tool_names": [
    "file_complaint",
    "search_complaints"
  ],
  "parameters": {
    "file_complaint": {
      "data": {
        "name": "Name of the person filing the complaint",
        "username": "User",
        "description": "Description of the complaint",
        "mobile_number": "1234567890"
      }
    },
    "search_complaints": {
      "data": {
        "username": "User",
        "mobile_number": "1234567890"
      }
    }
  }
}